In [1]:
# utilities to organise patches
class Node:
    __slots__ = ['left', 'right', 'value', 'idx']  # to occupy less memory, as tree will have many nodes

    def __init__(self, value, idx=None):
        self.left = None
        self.right = None
        self.value = value
        self.idx = idx

    def __repr__(self):
        return f"Node({self.value})"


class Tree:
    def __init__(self, criterion=lambda value, node_value: value < node_value):
        self.root = None
        self.nodes = []
        self.criterion = criterion  # criterion to assign to the left
        # speed up leaf computations, by starting computation from last computed leaves
        # this works assuming no nodes are removed from tree
        self._computed_leaves = []

    def get_root(self):
        return self.root

    def add(self, value):
        if self.root is None:
            new_node = Node(value, idx=len(self.nodes))
            self.root = new_node
            self.nodes.append(new_node)
        else:
            self.add_to_node(value, self.root)

    def add_to_node(self, value, node, left=None):
        if (left is not None and left) or self.criterion(value, node.value):
            if node.left is not None:
                self.add_to_node(value, node.left)
            else:
                new_node = Node(value, idx=len(self.nodes))
                node.left = new_node
                self.nodes.append(new_node)
        else:
            if node.right is not None:
                self.add_to_node(value, node.right)
            else:
                new_node = Node(value, idx=len(self.nodes))
                node.right = new_node
                self.nodes.append(new_node)

    def find(self, value):
        if self.root is not None:
            return self._find(value, self.root)
        else:
            return None

    def _find(self, value, node):
        if value == node.value:
            return node
        elif value < node.value and node.leaves is not None:
            self._find(value, node.leaves)
        elif value > node.value and node.right is not None:
            self._find(value, node.right)

    def delete_tree(self):
        # garbage collector will do this for us.
        self.root = None

    def get_leaves(self):
        leaves = []
        current_nodes = [self.root] if not self._computed_leaves else self._computed_leaves
        while len(current_nodes) > 0:
            next_nodes = []
            for node in current_nodes:
                if node.left is None and node.right is None:
                    leaves.append(node)
                    continue
                if node.left is not None:
                    next_nodes.append(node.left)
                if node.right is not None:
                    next_nodes.append(node.right)
            current_nodes = next_nodes
        self._computed_leaves = leaves
        return leaves

    def print_tree(self):
        if self.root is not None:
            self._print_tree(self.root)

    def _print_tree(self, node):
        if node is not None:
            self._print_tree(node.left)
            print(str(node.value) + ' ')
            self._print_tree(node.right)

    def __iter__(self):
        r"""Breadth first traversal"""
        current_nodes = [self.root]
        while len(current_nodes) > 0:
            next_nodes = []  # nodes in this list are labelled as 'discovered'
            for node in current_nodes:
                yield node.value
                if node.left is not None:
                    next_nodes.append(node.left)
                if node.right is not None:
                    next_nodes.append(node.right)
            current_nodes = next_nodes


In [6]:
import cv2
from numbers import Real

In [24]:
class TreeIterator:

    def __init__(self):
        self.spanning_trees = []
        self.tree_of_trees = Tree(lambda tree, value: tree.root.value[1] > value[1])

    def add_tree_for_region(self, region, stride, verbose=False):
        if len(region) == 4 and all(isinstance(dim, Real) for dim in region):  # if bouning box
            if verbose:
                print("Bounding box region ...")
            x, y, w, h = region
            region_tree = self.get_region_spanning_tree(stride, (x, y), limits=(x+w, y+h), verbose=verbose)
            self.spanning_trees.append(region_tree)
        else:
            if verbose:
                print("Contour region ...")
            x, y, w, h = cv2.boundingRect(region)
            stop_criterion = lambda point: cv2.pointPolygonTest(region, point, measureDist=False) >= 0
            region_tree = self.get_region_spanning_tree(stride, (x, y), stop_criterion=stop_criterion, verbose=verbose)
            self.spanning_trees.append(region_tree)
        self.update_tree_of_trees()

    @staticmethod
    def get_region_spanning_tree(stride, start, limits=None, stop_criterion=None, verbose=False):
        max_iter_num = 100
        if not bool(limits) ^ bool(stop_criterion):  # xor
            raise ValueError("Only one of limits or stop_criterion should be passed to function")
        tree = Tree(lambda value, node_value: value[1] > node_value[1])
        tree.add(tuple(start))
        nodes_were_added = True
        permanent_leaves = set()  # set of indices of nodes that should not grow anymore
        node_values = set()
        iter_num = 0
        if limits:
            while nodes_were_added:
                for leaf in tree.get_leaves():
                    if leaf.value[0] + stride < limits[0] and leaf.idx not in permanent_leaves:
                        new_point = (leaf.value[0] + stride, leaf.value[1])
                        if new_point in node_values:
                            permanent_leaves.add(len(tree.nodes))  # node about to be added will always be a leaf
                        tree.add_to_node(new_point, leaf, False)
                        node_values.add(new_point)
                        print(f"Iter {iter_num}: new point: {(leaf.value[0] + stride, leaf.value[1])}")
                    else:
                        nodes_were_added = False
                    if leaf.value[1] + stride < limits[1] and leaf.idx not in permanent_leaves:
                        new_point = (leaf.value[0], leaf.value[1] + stride)
                        if new_point in node_values:
                            permanent_leaves.add(len(tree.nodes))  # node about to be added will always be a leaf
                        tree.add_to_node((leaf.value[0], leaf.value[1] + stride), leaf, True)
                        node_values.add(new_point)
                        nodes_were_added = True
                        if verbose:
                            print(f"Iter {iter_num}: new point: {(leaf.value[0], leaf.value[1] + stride)} - num perm leaves {len(permanent_leaves)}")
                iter_num += 1
                if iter_num > max_iter_num:
                    raise ValueError("Max number iterations reached")
        else:
            raise ValueError("Either limits or stop_criterion must not be None")
        return tree

    def update_tree_of_trees(self):
        tree_of_trees = Tree(lambda added_tree, node_tree: tree.root.value[1] > node_tree.root.value[1])
        for tree in self.spanning_trees:
            tree_of_trees.add(tree)
        self.tree_of_trees = tree_of_trees

    def __iter__(self):
        for tree in self.tree_of_trees:
            yield from tree


In [25]:
ti = TreeIterator()

In [34]:
ti.add_tree_for_region((0, 0, 10000, 10000), 1000, True)

Bounding box region ...
Iter 0: new point: (1000, 0)
Iter 0: new point: (0, 1000) - num perm leaves 0
Iter 1: new point: (1000, 1000)
Iter 1: new point: (0, 2000) - num perm leaves 0
Iter 1: new point: (2000, 0)
Iter 1: new point: (1000, 1000) - num perm leaves 1
Iter 2: new point: (1000, 2000)
Iter 2: new point: (0, 3000) - num perm leaves 1
Iter 2: new point: (2000, 1000)
Iter 2: new point: (1000, 2000) - num perm leaves 2
Iter 2: new point: (3000, 0)
Iter 2: new point: (2000, 1000) - num perm leaves 3
Iter 3: new point: (1000, 3000)
Iter 3: new point: (0, 4000) - num perm leaves 3
Iter 3: new point: (2000, 2000)
Iter 3: new point: (1000, 3000) - num perm leaves 4
Iter 3: new point: (3000, 1000)
Iter 3: new point: (2000, 2000) - num perm leaves 5
Iter 3: new point: (4000, 0)
Iter 3: new point: (3000, 1000) - num perm leaves 6
Iter 4: new point: (1000, 4000)
Iter 4: new point: (0, 5000) - num perm leaves 6
Iter 4: new point: (2000, 3000)
Iter 4: new point: (1000, 4000) - num perm leav

In [35]:
len(ti.spanning_trees[-1].nodes)
set(ti.spanning_trees[-1].nodes)

{Node((0, 0)),
 Node((0, 1000)),
 Node((0, 2000)),
 Node((0, 3000)),
 Node((0, 4000)),
 Node((0, 5000)),
 Node((0, 6000)),
 Node((0, 7000)),
 Node((0, 8000)),
 Node((0, 9000)),
 Node((1000, 0)),
 Node((1000, 1000)),
 Node((1000, 1000)),
 Node((1000, 2000)),
 Node((1000, 2000)),
 Node((1000, 3000)),
 Node((1000, 3000)),
 Node((1000, 4000)),
 Node((1000, 4000)),
 Node((1000, 5000)),
 Node((1000, 5000)),
 Node((1000, 6000)),
 Node((1000, 6000)),
 Node((1000, 7000)),
 Node((1000, 7000)),
 Node((1000, 8000)),
 Node((1000, 8000)),
 Node((1000, 9000)),
 Node((1000, 9000)),
 Node((2000, 0)),
 Node((2000, 1000)),
 Node((2000, 1000)),
 Node((2000, 2000)),
 Node((2000, 2000)),
 Node((2000, 3000)),
 Node((2000, 3000)),
 Node((2000, 4000)),
 Node((2000, 4000)),
 Node((2000, 5000)),
 Node((2000, 5000)),
 Node((2000, 6000)),
 Node((2000, 6000)),
 Node((2000, 7000)),
 Node((2000, 7000)),
 Node((2000, 8000)),
 Node((2000, 8000)),
 Node((2000, 9000)),
 Node((2000, 9000)),
 Node((3000, 0)),
 Node((3000, 

In [32]:
ti.spanning_trees[-1].root.right.left.left